# Set Game

## Apresentação 

O Set Game é um jogo de cartas projetado por Marsha Falco em 1974 que consiste em um baralho de 81 cartas únicas que possuem 4 caracaterísticas cada: um símbolo, a cor desse símbolo, o estilo de preenchimento desse símbolo e o número de repetições desse símbolo. Cada uma dessas características podem assumir 3 valores distintos. A imagem abaixo exemplifica 3 cartas diferentes e as características possíveis: 

![image](cards_example.png)

Nessa imagem podemos ver os possíveis valores de cada característica:

- __Símbolo__: diamante, curvado e oval;

- __Cor do símbolo__: verde, roxo e vermelho;

- __Estilo de preenchimento__: vazio, listrado e completo;

- __Quantidade de símbolos__: um, dois ou três;

Determinadas combinações de 3 cartas podem formar um conjunto válido (__set__), para isso ocorrer, cada uma das características dessas três cartas deve ser igual em todas as cartas, ou diferente em cada uma delas. A imagem acima é um conjunto válido, pois as 3 cartas possuem os símbolos distintos, a cores distintas, o estilo de preenchimento distinto e a quantidade de símbolos distinta. Caso a última carta fosse roxa e não vermelha, não teríamos um conjunto válido, pois teríamos uma carta verde e duas cartas roxas (precisamos que sejam as três iguais ou três distintas).

O objetivo do jogo é formar esses conjuntos válidos a partir de um grupo de cartas aleatório, uma versão online desse jogo está disponível no site [The Daily SET Puzzle](https://www.setgame.com/set/puzzle), nele diariamente são sorteadas 12 cartas e o objetivo é formar 6 conjuntos válidos distintos. Nesse trabalho iremos modelar esta versão do jogo através de programação linear e inteira.

## Formulação do problema

In [21]:
using JuMP
using Cbc

Cada carta será representada por um vetor de dimensão 4, cada uma dessas dimensões representará uma característica da carta: cor, símbolo, preenchimento, e o número de símbolos.

Cada uma dessas características pode assumir três valores distintos, para cada uma delas iremos representar os valores através dos números de 1 a 3:

- __Cor__ : 1 - vermelho; 2 - verde; 3 - roxo;

- __Símbolo__: 1 - curvado; 2 - diamante; 3 - oval;

- __Preenchimento__: 1 - completo; 2 - listrado; 3 - vazio;

- __Número__: 1; 2; 3; (vai se manter)

Dessa forma, seja $x$ uma carta, temos que $x \in \{1, 2, 3\}^4$. O objetivo do jogo é dado um conjunto de 12 cartas, formar 6 conjuntos válidos distintos que possuem cada um 3 cartas. Um conjunto válido de 3 cartas é um conjunto tal que seja $x$, $y$, $z$ essas cartas, temos que:
$$x_i = y_i = z_i \quad \text{ou} \quad (x_i \neq y_i \land y_i \neq z_i \land x_i \neq z_i)  \quad \forall i \in \{1, 2, 3, 4\} $$

Ou seja, para cada uma das características da cada, ou as 3 cartas possuem essa característica igual, ou as 3 cartas possuem diferentes características.

O nosso objetivo é encontrar as cartas que formas os 6 conjuntos válidos (distintos) dentre as 12 cartas possíveis. 

Criamos a variável $C_{6\times 3 \times 4}$, a primeira dimensão representa os 6 conjuntos distintos, a segunda dimensão representa as 3 cartas distintas de cada conjunto, e a terceira dimensão representa as 4 características de cada carta.

In [22]:
setgame = Model(Cbc.Optimizer) 
@variable(setgame, C[1:6, 1:3, 1:4]);

Vamos representar por $c_{ij}$ a $j$-ésima carta pertencente ao $i$-ésimo conjunto válido. Para cada $c_{ij}$ criamos 12 variáveis binárias $b_{ijk}, k \in \{1, \dots, 12\}$ que serão utilizadas para indicar que a carta deve ser igual a pelo menos uma das 12 cartas disponíveis. Seja $D$ o nosso conjunto de cartas disponíveis (com alguma ordem), temos:
$$c_{ij} == \sum_{k = 1}^{12} D_k b_{ijk} \quad \forall k \in \{1, \dots, 12\}$$
$$\sum_{k = 1}^{12} b_{ijk} = 1$$
$$b_{ijk} \text{ binárias} \quad \forall k \in \{1, \dots, 12\}$$

Iniciante vamos criar o conjunto $D$ usando as cartas possíveis do dia 03/04/2021:

In [23]:
HTML("""
<img src = "possible_cards_03_04.png"></img>
""")

HTML{String}("<img src = \"possible_cards_03_04.png\"></img>\n")

In [24]:
#possible cards in 03/04/2021
D = [[3 1 1 2];
    [1 3 2 1];
    [3 3 2 2];
    [1 1 1 3];
    [1 1 3 1];
    [2 2 3 2];
    [3 3 3 2];
    [3 3 1 2];
    [1 2 1 1];
    [2 2 2 1];
    [1 1 2 2];
    [3 2 3 1]];

Agora definimos as variáveis binárias:

In [25]:
@variable(setgame, b[1:6, 1:3, 1:12], binary = true)
#for each i,j the sum b_ijk is 1
@constraint(setgame, sum_of_b[i = 1:6, j = 1:3], sum(b[i, j, :]) == 1)

#equality constraints
@constraint(setgame, equality_b[i = 1:6, j = 1:3, k = 1:4], C[i, j, k] == sum(D .* b[i, j, :], dims = 1)[k]);

Vamos denominar por $c_{ijk}$ a k-ésima característica, da j-ésima carta, do i-ésimo conjunto. O segundo grupos de restrições que devemos considerar é que os conjuntos são válidos, isto é, para cada $i$, cada uma das dimensões das cartas $c_{i1}, c_{i2}, c_{i3}$ ou são todas iguais ou são todas distintas. Pela dificuldade de expressar as relações de "ou" e "e" presentes nessas restrições, vamos listar todas as combinações possíveis que cada coordenada das 3 cartas podem assumir, criando o conjunto $E$.

In [26]:
E = [[1 1 1];
    [2 2 2];
    [3 3 3];
    [1 2 3];
    [1 3 2];
    [2 1 3];
    [2 3 1];
    [3 1 2];
    [3 2 1]];

Da mesma forma que fizemos que as cartas deveriam assumir algum valor de $D$ através das variáveis binárias $b_{ijk}$, para cada conjunto $i$ e para cada característica $k$ vamos definir as variáveis binárias $f_{ikl}, l \in \{1, \dots, 9\}$ que serão utilizadas para indicar que as características devem assumir pelo menos alguma combinação presente em $E$.

$$(c_{i1k}, c_{i2k}, c_{i3k}) == \sum_{l = 1}^{9} E_k f_{ikl} \quad \forall l \in \{1, \dots, 9\}$$
$$\sum_{l = 1}^{9} f_{ikl} = 1$$
$$f_{ikl} \text{ binárias} \quad \forall l \in \{1, \dots, 9\}$$

In [27]:
@variable(setgame, f[1:6, 1:4, 1:9], binary = true)
#for each i,k the sum f_ikl is 1
@constraint(setgame, sum_of_f[i = 1:6, k = 1:4], sum(f[i, k, :]) == 1)

#equality constraints
@constraint(setgame, equality_f[i = 1:6, j = 1:3, k = 1:4], C[i, j, k] == sum(E .* f[i, k, :], dims = 1)[j]);

Outra regra que torna o conjunto válido que se deve levar em conta, é que todas as cartas devem ser distintas (pois qualquer repetição de 3 vezes a mesma carta gera um conjunto válido), para isso retornaremos para as nossas variáveis $b_{ijk}$, para cada conjunto $i$, queremos que as variáveis $b_{i1k}$, $b_{i2k}$ e $b_{i3k}$ assumam valores iguais a $1$ para distintos valores de $k$, ou seja:
$$\text{para cada } i, k$$
$$\sum_{j = 1}^3 b_{ijk} \leq 1$$

In [28]:
#sets with different cards
@constraint(setgame, different_cards[i = 1:6, k = 1:12], sum(b[i, :, k]) <= 1);

O último grupo de restrições que devemos considerar é que cada conjunto de cartas deve ser distinto um do outro, para cada par de conjunto de cartas $i$ e $i'$ temos que as $3$ cartas de cada um dos conjuntos devem ser todas distintas, para isso usaremos novamente as variáveis $b_{ijk}$. Como não sabemos a ordem das cartas em cada conjunto de cartas, devemos comparar as 3 cartas do conjunto $i$ com as 3 cartas do conjunto $i'$ individualmente, tendo então 9 comparações, e pelo menos 7 das desigualdades devem valer, para isso utilizaremos variáveis binárias $g_{ii'jj'}$ que indicam quais das desigualdades valem e a soma delas deverá ser maior ou igual a 7. Para comparar duas cartas, iremos utilizar a mesma forma que a restrição anterior, as somas de $b_{ijk}$ deve ser menor ou igual a 1.
$$ \text{para cada } i, i'  \text{distintos} $$
$$ \text{para cada } j, j' \in \{1, 2, 3\}$$
$$ \text{para cada k} $$
$$ b_{ijk} + b_{i'j'k} \leq 1 + 100(1 - g_{ii'jj'}) $$
$$\sum_{j = 1}^{3} \sum_{j' = 1}^3 g_{ii'jj'} \geq 7$$

In [29]:
for i = 1:6
    for i_line = (i+1):6
        g = @variable(setgame, [j = 1:3, j_line = 1:3], binary = true)
        @constraint(setgame, sum(g) >= 7)
        for j = 1:3
            for j_line = 1:3
                @constraint(setgame, [k = 1:12], b[i,j,k] + b[i_line, j_line, k] <= 1 + 100(1 - g[j, j_line]))
                
            end
        end
    end
end

In [30]:
optimize!(setgame)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 0 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 77 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 42 strengthened rows, 0 substitutions
Cgl0004I processed model has 1821 rows, 567 columns (567 integer (567 of which binary)) and 7269 elements
Cbc0045I No integer variables out of 567 objects (567 integer) have costs
Cbc0045I branch on satisfied N create fake objective Y random cost Y
Cbc0031I 82 added rows had average density of 56.02439
Cbc0013I At root node, 82 cuts changed objective from 0 to 0 in 10 passes
Cbc0014I Cut generator 0 (Probing) - 102 row cuts average 3.9 elements, 0 column cuts (35 active)  in 0.034 seconds - new frequency is -100
Cbc0014I Cut generator 1 (Gomory) - 473 row cuts average 215.6 elements, 0 column cuts (0 active)  in 0.066 seconds - new frequency is -100
Cbc

In [31]:
result = JuMP.value.(C)

6×3×4 Array{Float64,3}:
[:, :, 1] =
 3.0  3.0  3.0
 2.0  1.0  3.0
 2.0  3.0  1.0
 1.0  1.0  1.0
 1.0  3.0  2.0
 1.0  1.0  1.0

[:, :, 2] =
 3.0  3.0  3.0
 2.0  1.0  3.0
 2.0  3.0  1.0
 1.0  1.0  1.0
 2.0  2.0  2.0
 3.0  1.0  2.0

[:, :, 3] =
 1.0  2.0  3.0
 2.0  1.0  3.0
 3.0  1.0  2.0
 3.0  2.0  1.0
 1.0  3.0  2.0
 2.0  3.0  1.0

[:, :, 4] =
 2.0  2.0  2.0
 1.0  3.0  2.0
 2.0  2.0  2.0
 1.0  2.0  3.0
 1.0  1.0  1.0
 1.0  1.0  1.0

In [32]:
for i = 1:6
    display(result[i, :, :])
end

3×4 Array{Float64,2}:
 3.0  3.0  1.0  2.0
 3.0  3.0  2.0  2.0
 3.0  3.0  3.0  2.0

3×4 Array{Float64,2}:
 2.0  2.0  2.0  1.0
 1.0  1.0  1.0  3.0
 3.0  3.0  3.0  2.0

3×4 Array{Float64,2}:
 2.0  2.0  3.0  2.0
 3.0  3.0  1.0  2.0
 1.0  1.0  2.0  2.0

3×4 Array{Float64,2}:
 1.0  1.0  3.0  1.0
 1.0  1.0  2.0  2.0
 1.0  1.0  1.0  3.0

3×4 Array{Float64,2}:
 1.0  2.0  1.0  1.0
 3.0  2.0  3.0  1.0
 2.0  2.0  2.0  1.0

3×4 Array{Float64,2}:
 1.0  3.0  2.0  1.0
 1.0  1.0  3.0  1.0
 1.0  2.0  1.0  1.0

In [33]:
b_result = JuMP.value.(b);

In [34]:
sum(D .* b_result[1, 1, :], dims = 1)

1×4 Array{Float64,2}:
 3.0  3.0  1.0  2.0